In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm

sns.set()

In [ ]:
df = pd.read_csv('Folkhalsomyndigheten.csv',sep=';',encoding='ISO-8859-1')
df['Region'].replace('Jämtland Härjedalen','Jämtland',inplace=True)
df['Region'].replace('Sörmland','Södermanland',inplace=True)
df.set_index('Region',inplace=True)
df

In [ ]:
pop_df = pd.read_csv('sverige_befolkning.csv',sep=';',thousands=' ',header=None,usecols=[0,1,2])
pop_df.set_index(0,inplace=True)
pop_df

In [ ]:
def regioner(x):
    if len(x) == 2:
        return 'region'
    else:
        return 'kommun'

grouped = pop_df.groupby(regioner)

region_df = grouped.get_group('region')
region_df = region_df.set_index(1)
region_df.rename(columns={2:'befolkning'},inplace=True)
region_df.index.name='region'
region_df

In [ ]:
df = df.join(region_df)
df.drop('Fall_per_100000_inv',axis=1,inplace=True)
df.columns=['confirmed','icu','dead','population']
df['conf_per_M'] = df['confirmed'] / (df['population'] / 1e6)
df['dead_per_M'] = df['dead'] / (df['population'] / 1e6)
df.sort_values('dead_per_M',inplace=True,ascending=False)
df

In [ ]:
print (df.describe())
df.sum()

In [ ]:
df_age = pd.read_csv('Folkhalsomyndigheten_age.csv',sep=';',encoding='ISO-8859-1')
df_age

In [ ]:
df_age.plot(kind='bar',x='Åldersgrupp',y=['Totalt_antal_fall','Totalt_antal_avlidna'],figsize=(18,12))
plt.title('Corona Sverige åldersprofil')
plt.ylabel('Antal')
plt.savefig('Corona_sverige_age.jpg',format='jpg')

In [ ]:
df['conf_of_total'] = 100 * df['confirmed'] / df['confirmed'].sum()
df['dead_of_total'] = 100 * df['dead'] / df['dead'].sum()

df.plot(kind='bar',y=['conf_of_total','dead_of_total'],figsize=(18,12))
plt.title('Corona Sweden - percentages confirmed & dead per Region')
plt.ylabel('percent')
plt.savefig('corona_sweden_regions_pct.jpg',format='jpg')